In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from fastcoref import TrainingArgs, CorefTrainer
import spacy
import spacy_pythainlp.core
from fastcoref import FCoref

In [3]:
nlp = spacy.blank("th")

In [4]:
model = FCoref(
    model_name_or_path='xlm-v-base-model/model',
    device='cuda',
    nlp=nlp
)

05/21/2023 06:53:14 - INFO - 	 missing_keys: []
05/21/2023 06:53:14 - INFO - 	 unexpected_keys: []
05/21/2023 06:53:14 - INFO - 	 mismatched_keys: []
05/21/2023 06:53:14 - INFO - 	 error_msgs: []
05/21/2023 06:53:14 - INFO - 	 Model Parameters: 786.9M, Transformer: 778.5M, Coref head: 8.4M


In [5]:
preds = model.predict(
   texts=["แกว่าเราเข้มแข็งมากพอที่จะเห็นแกไปน่ารักกับใครหรอ"]
)

05/21/2023 06:53:14 - INFO - 	 Tokenize 1 inputs...


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

05/21/2023 06:53:14 - INFO - 	 ***** Running Inference on 1 texts *****


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
preds[0].get_clusters(as_strings=True)

[['แก', 'แก']]

In [7]:
preds[0].get_clusters(as_strings=False)

[[(0, 2), (30, 32)]]

In [8]:
import json

In [29]:
with open("dataset/val_2152023_newmm.jsonlines","r",encoding="utf-8-sig") as f:
    data=[eval(i.strip()) for i in f.readlines()]

In [30]:
data[2]

{'text': 'แม่และพ่อน้อง จีจี้ เน็ตไอดอลดัง ร่ำไห้รับศพไปทำพิธีทางศาสนาที่นครปฐม แม่เผยไม่อยากให้ลูกคบแฟนหนุ่ม แต่ทั้งคู่ยังดื้อจะรักกันให้ได้',
 'clusters': [[[0, 3], [70, 73]], [[9, 32], [86, 89]]],
 'clusters_strings': [['แม่', 'แม่'], ['น้อง จีจี้ เน็ตไอดอลดัง', 'ลูก']],
 'source': 'Thai news from thairath.com (Politics news)'}

In [31]:
texts = [i['text'] for i in data]

In [32]:
preds = model.predict(
   texts=texts
)

05/21/2023 06:58:41 - INFO - 	 Tokenize 150 inputs...


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

05/21/2023 06:58:42 - INFO - 	 ***** Running Inference on 150 texts *****


Inference:   0%|          | 0/150 [00:00<?, ?it/s]

In [33]:
preds[0].get_clusters(as_strings=True)

[['เมย์', 'ตัวเอง']]

In [34]:
preds[0].get_clusters(as_strings=False)

[[(0, 4), (45, 51)]]

In [35]:
len(preds)

150

In [36]:
type(preds)

list

In [37]:
i=53

In [38]:
preds[i]

CorefResult(text="บิ๊กโจ๊ก ระบุ ตร.มีหลักฐาน ตู้ห่าว เข้าออกผับจินหล...", clusters=[['ตู้ห่าว', 'นายตู้ห่าว']])

In [39]:
data[i]

{'text': 'บิ๊กโจ๊ก ระบุ ตร.มีหลักฐาน ตู้ห่าว เข้าออกผับจินหลิงบ่อย และเป็นผู้เช่าอาคาร เชื่อได้ว่านายตู้ห่าวเป็นผู้อยู่เบื้องหลังในการสั่งการและบริหาร ผับจินหลิง',
 'clusters': [[[0, 8], [14, 17]], [[27, 34], [88, 98]]],
 'clusters_strings': [['บิ๊กโจ๊ก', 'ตร.'], ['ตู้ห่าว', 'นายตู้ห่าว']],
 'source': 'Thai news (news)'}

In [40]:
correct_all = 0

In [41]:
for p,r in zip(preds,data):
    if p.get_clusters(as_strings=True)==r['clusters_strings']:
        correct_all+=1
    else:
        print(r['text'])
        print("Ref: "+str(r['clusters_strings']))
        print("Pred: "+str(p.get_clusters(as_strings=True)))
        print()

แฟนคลับอาลัย อิงดาว ดุจดาริกา นางเอกหมอลำเสียงวิหค เสียชีวิต บอกเธอจะอยู่ในความทรงจำของพวกเราตลอดไป
Ref: [['แฟนคลับ', 'พวกเรา'], ['อิงดาว ดุจดาริกา', 'เธอ']]
Pred: [['อิงดาว ดุจดาริกา', 'เธอ']]

แม่และพ่อน้อง จีจี้ เน็ตไอดอลดัง ร่ำไห้รับศพไปทำพิธีทางศาสนาที่นครปฐม แม่เผยไม่อยากให้ลูกคบแฟนหนุ่ม แต่ทั้งคู่ยังดื้อจะรักกันให้ได้
Ref: [['แม่', 'แม่'], ['น้อง จีจี้ เน็ตไอดอลดัง', 'ลูก']]
Pred: [['แม่', 'แม่'], ['น้อง จีจี้', 'ลูก']]

จากประเด็นที่ พล.อ.ประยุทธ์ จันทร์โอชา นายกรัฐมนตรี กล่าวปาฐกถาในงานประชุมสมัชชาสหประชาชาติ ที่นิวยอร์ก และพูดถึงประเด็นคนไทยและการใช้งานกูเกิล โดย พล.อ.ประยุทธ์ บอกว่า คนไทยไม่ค่อยใช้งานกูเกิลกัน จนชาวโซเชียลวิจารณ์กันมากมาย ล่าสุด พล.อ.ประยุทธ์ ออกมาตอบโต้ว่า ไม่ได้บอกว่าคนไทยไม่ใช้กูเกิลหรือใช้ไม่เป็น และยังบอกด้วยว่าคนเอาไปบิดเบือน ทำไมตนจะไม่รู้ข้อมูลว่าคนไทยใช้โทรศัพท์เท่าไรในประเทศไทยหรือใช้กูเกิลกันเท่าไร ตนแค่พูดกับคนไทยในสหรัฐฯที่ถามว่าเขาจะรู้เรื่องราวในประเทศไทยได้อย่างไร ตนก็บอกให้พวกเขาเปิดกูเกิลค้นหาดู ตนพูดแค่นี้ แต่คนเอาไปขยายความกัน ทำให้ทุกอย่

In [42]:
correct_all

88